In [38]:
import pandas as pd
import numpy as np

In [20]:
df = pd.read_csv('../res/datasets/all_transactions.csv')
df.head()

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date
0,10/28/2022,Joint,MSFT,Microsoft Corporation - Common Stock <div clas...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022
1,10/31/2022,Joint,CLF,Cleveland-Cliffs Inc. Common Stock,Stock,Purchase,"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022
2,10/28/2022,Joint,MSFT,Microsoft Corporation - Common Stock <div clas...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022
3,10/28/2022,Joint,MSFT,Microsoft Corporation - Common Stock <div clas...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022
4,10/24/2022,Joint,MSFT,Microsoft Corporation - Common Stock,Stock,Purchase,"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022


In [21]:
df.columns

Index(['transaction_date', 'owner', 'ticker', 'asset_description',
       'asset_type', 'type', 'amount', 'comment', 'senator', 'ptr_link',
       'disclosure_date'],
      dtype='object')

In [22]:
len(df)

9680

In [23]:
# earliest trade recorded
df.iloc[len(df) - 1]['transaction_date']

'7/25/2012'

In [24]:
df = df[df['type'] == 'Purchase']
df = df[df['asset_type'] == 'Stock']
df = df.drop(['comment', 'ptr_link'], axis=1)
df

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,senator,disclosure_date
1,10/31/2022,Joint,CLF,Cleveland-Cliffs Inc. Common Stock,Stock,Purchase,"$15,001 - $50,000",Thomas H Tuberville,11/10/2022
4,10/24/2022,Joint,MSFT,Microsoft Corporation - Common Stock,Stock,Purchase,"$15,001 - $50,000",Thomas H Tuberville,11/10/2022
5,10/13/2022,Joint,NU,Nu Holdings Ltd. Class A Ordinary Shares,Stock,Purchase,"$15,001 - $50,000",Thomas H Tuberville,11/10/2022
6,10/13/2022,Joint,MSFT,Microsoft Corporation - Common Stock,Stock,Purchase,"$250,001 - $500,000",Thomas H Tuberville,11/10/2022
7,10/11/2022,Joint,NU,Nu Holdings Ltd. Class A Ordinary Shares,Stock,Purchase,"$1,001 - $15,000",Thomas H Tuberville,11/10/2022
...,...,...,...,...,...,...,...,...,...
8749,12/10/2014,Spouse,MDDVX,BlackRock Equity Dividend Inv A (NASDAQ),Stock,Purchase,"$1,001 - $15,000",Susan M Collins,1/9/2015
8750,12/11/2015,Spouse,DVFAX,Cohen &amp; Steers Dividend Value A (NASDAQ),Stock,Purchase,"$1,001 - $15,000",Susan M Collins,1/9/2015
8751,12/11/2014,Spouse,DVFAX,Cohen &amp; Steers Dividend Value A (NASDAQ),Stock,Purchase,"$1,001 - $15,000",Susan M Collins,1/9/2015
8758,12/19/2014,Spouse,BA,The Boeing Company (NYSE),Stock,Purchase,"$1,001 - $15,000",Pat Roberts,1/5/2015


In [26]:
len(df)

3420

In [33]:
def fetchSession(url):
    session = HTMLSession()
    r = session.get(url)
    return r

def value_to_ints(value):
    bad_chars = [
        ',','$','-'
    ]
    for c in bad_chars:
        value = value.replace(c,'')
    low, high = [
        int(x) for x in (value.split('  ', 1))
    ]
    return [low,high]

def getYahooInfo(ticker):
    url = 'https://finance.yahoo.com/quote/{}'.format(ticker)
    r = fetchSession(url)
    # handle invalid ticker
    tables = r.html.find('table')
    if len(tables) == 1:
        return -1,-1
    
    left_table = tables[0]
    right_table = tables[1]
    left_rows = left_table.find('td')
    right_rows = right_table.find('td')
    left_items = []
    left_values = []
    right_items = []
    right_values = []
    
    i = 0
    for l,r in zip(left_rows, right_rows):
        # evens = item headers
        if i % 2 == 0:
            left_items.append(l.text)
            right_items.append(r.text)
        # odds = values in table
        else:
            left_values.append(l.text)
            right_values.append(r.text)
        i += 1
    return (
        dict(
            zip(left_items, left_values)
        ),
        dict(
            zip(right_items, right_values)
        )
    )

In [32]:
i = 0
for i in range(len(df)):
    row = df.iloc[i]
    ticker = row['ticker']
    value = value_to_ints(row['amount'])[1]
    

CLF 50000
MSFT 50000
NU 50000
MSFT 500000
NU 15000
NU 15000
VCTR 15000
VCTR 15000
VCTR 15000
VRTX 15000
TREX 15000
SCI 15000
RELX 15000
RH 15000
PHM 15000
PAYC 15000
GIS 15000
GLPI 15000
RACE 15000
XOM 100000
EXR 15000
ETSY 15000
ELS 15000
DQ 15000
CROX 15000
CSCO 15000
AMGN 15000
AMT 15000
BMN 15000
CVS 15000
TSM 15000
MPW 15000
ABT 15000
X 250000
MSFT 500000
ARKK 15000
ARKK 100000
NU 15000
WFC 15000
ABNFX 50000
RSP 50000
IDV 50000
DHS 50000
MUB 100000
OC 50000
ARKK 100000
INTC 15000
INTC 250000
SQM 15000
CLF 100000
ECOM 250000
GWW 15000
COST 15000
CMS 15000
IBDO 15000
IBDQ 50000
PYPL 100000
X 250000
SSYS 15000
INTC 100000
PYPL 250000
QCOM 500000
PYPL 15000
SSYS 15000
SSYS 50000
INTC 50000
EVT 50000
ECOM 15000
WFC 15000
X 250000
PYPL 100000
PYPL 500000
FSK 15000
PYPL 50000
ECOM 250000
LMST 15000
KEY 15000
FGBI 15000
FITB 15000
CFG 15000
BMO 15000
TMO 15000
RMD 15000
RF 15000
NOC 15000
NDAQ 15000
MRK 15000
MASI 15000
MMC 15000
MKSI 15000
HUMA 50000
HON 15000
GFI 15000
IT 15000
DHI 1500